# Most Used Functions in LSGAN

Least Squares GAN (LSGAN) is a type of Generative Adversarial Network (GAN) that uses least squares loss for the discriminator. This helps to address issues with the original GAN formulation, such as vanishing gradients and instability during training. In this notebook, we will cover some of the most commonly used functions and techniques for implementing a simplified version of LSGAN using TensorFlow and Keras.

## 1. Building the Generator

The generator in LSGAN generates images from random noise. It uses a series of transposed convolutional layers to upsample the input noise vector.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Conv2DTranspose, BatchNormalization, Activation
from tensorflow.keras.models import Model

# Function to build the generator
def build_generator():
    inputs = Input(shape=(100,))
    x = Dense(256 * 7 * 7, activation='relu')(inputs)
    x = Reshape((7, 7, 256))(x)
    x = Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(1, (5, 5), strides=(1, 1), padding='same')(x)
    x = Activation('tanh')(x)
    return Model(inputs, x)

# Instantiate and summarize the generator
generator = build_generator()
generator.summary()

## 2. Building the Discriminator

The discriminator in LSGAN distinguishes between real and fake images. It uses a series of convolutional layers to downsample the input image.

In [ ]:
from tensorflow.keras.layers import Conv2D, LeakyReLU, Flatten

# Function to build the discriminator
def build_discriminator():
    inputs = Input(shape=(28, 28, 1))
    x = Conv2D(64, (5, 5), strides=(2, 2), padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, (5, 5), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)
    x = Dense(1)(x)  # No activation for least squares loss
    return Model(inputs, x)

# Instantiate and summarize the discriminator
discriminator = build_discriminator()
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='mse', metrics=['accuracy'])
discriminator.summary()

## 3. Building the LSGAN

The LSGAN combines the generator and discriminator. The generator tries to generate realistic images, while the discriminator distinguishes between real and fake images.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Function to build the LSGAN
def build_lsgan(generator, discriminator):
    discriminator.trainable = False
    inputs = Input(shape=(100,))
    generated_image = generator(inputs)
    validity = discriminator(generated_image)
    model = Model(inputs, validity)
    model.compile(optimizer=Adam(0.0002, 0.5), loss='mse')
    return model

# Instantiate and summarize the LSGAN
lsgan = build_lsgan(generator, discriminator)
lsgan.summary()

## 4. Training the LSGAN

Training the LSGAN involves alternating between training the discriminator and training the generator using least squares loss.

In [ ]:
# Load and preprocess the data
(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
X_train = np.expand_dims(X_train, axis=-1)

# Training parameters
epochs = 10000
batch_size = 64
half_batch = batch_size // 2

# Training loop
for epoch in range(epochs):
    # Train discriminator with real samples
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    X_real, y_real = X_train[idx], np.ones((half_batch, 1))
    d_loss_real = discriminator.train_on_batch(X_real, y_real)
    
    # Train discriminator with fake samples
    noise = np.random.normal(0, 1, (half_batch, 100))
    X_fake = generator.predict(noise)
    y_fake = np.zeros((half_batch, 1))
    d_loss_fake = discriminator.train_on_batch(X_fake, y_fake)
    
    # Train generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    y_gan = np.ones((batch_size, 1))
    g_loss = lsgan.train_on_batch(noise, y_gan)
    
    # Summarize the loss for this epoch
    if (epoch + 1) % 1000 == 0:
        print(f'{epoch+1}/{epochs} [D real: {d_loss_real:.3f}] [D fake: {d_loss_fake:.3f}] [G loss: {g_loss:.3f}]')